In [27]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import statistics

In [53]:
# Given array of equities, returning dataframe with historical data dating back 
def get_closing_price_historical_data(equities, period, interval):
    data = {}
    for x in equities:
        ticker = yf.Ticker(x)
        history = ticker.history(period= period, interval= interval)['Close']
        data[x] = history
    return data

# Given a m x n matrix  of m closing prices for n equities, returns an 1 x n 
# array of average closing prices
def compute_average(closing_prices_for_multiple_equities):
    # print(closing_prices_for_multiple_equities)
    sums = [0] * len(closing_prices_for_multiple_equities[0])
    for row in closing_prices_for_multiple_equities:
        for i in range(len(row)):
            sums[i] += row[i]

    avgs = [x/len(closing_prices_for_multiple_equities) for x in sums]
    return avgs

# Given an m x n matrix of m closing prices for n equities and n averages for n 
# equities, returns all of the difference of the closing prices and their averages
# essentially substracts mean to make the new mean of the data set 0 for each column
def compute_stocks_demeaned(closing_prices_matrix, average_closing_prices):
    m = len(closing_prices_matrix)
    n = len(closing_prices_matrix[0])
    demeaned = [[0 for i in range(n)] for j in range(m)]
    for i in range(m):
        for j in range(n):
            demeaned[i][j] = closing_prices_matrix[i][j] - average_closing_prices[j]

    return demeaned

# Given an m x n matrix of the demeaned closing prices for n equities, returns
# the associated covariance matrix calculated by demeaned transpose x demeaned.
def compute_covariance_matrix(demeaned):
    s_minus_m = np.array(demeaned)
    s_minus_m_t = s_minus_m.transpose()
    m = len(demeaned)
    return np.matmul(s_minus_m_t, s_minus_m)/m

def expected_portfolio_risk_based_on_stddev (asset_weights, covariance_matrix):
    Wt = np.asmatrix(asset_weights)
    W = Wt.transpose()
    return np.sqrt(Wt.dot(covariance_matrix).dot(W))

def expected_portfolio_return(average_closing_prices, asset_weights):
    M = np.array(average_closing_prices)
    W = np.array(asset_weights)
    return M.dot(W)

def sharpe_ratio(closing_prices_matrix, asset_weights, risk_free_rate):
    average_closing_prices = compute_average(closing_prices_matrix)
    
    demeaned = compute_stocks_demeaned(closing_prices_matrix, average_closing_prices)
    cov = compute_covariance_matrix(demeaned)
    risk = expected_portfolio_risk_based_on_stddev(asset_weights, cov)
    ret = expected_portfolio_return(average_closing_prices, asset_weights)
    print(risk, ret, risk_free_rate)
    return (ret - risk_free_rate)/risk

In [29]:
trimmed_senbet = pd.read_excel('Senbet Portfolio.xlsx')
trimmed_senbet.columns = ['Ticker', 'Allocation', 'Shares']
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.O', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.K', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace(' ', ''), axis = 1)
asset_weights = list(trimmed_senbet['Allocation'])

In [30]:
historical_data = get_closing_price_historical_data(trimmed_senbet['Ticker'], '200d', '1d')

In [34]:
portfolio = pd.DataFrame(historical_data)
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-23,194.046616,2986.550049,109.139999,53.188820,12.578026,123.918304,176.338181,192.905685,315.196838,143.453598,...,69.200691,728.419434,165.299011,99.192329,16.188866,95.025688,199.918991,95.644043,38.295872,33.586208
2020-07-24,195.195847,3008.909912,110.180000,52.690704,12.429231,123.721855,177.175522,190.570129,314.877899,143.602936,...,69.023354,729.111633,165.328445,98.226616,15.878473,95.491646,198.392654,94.318268,38.129070,33.189907
2020-07-27,197.680984,3055.209961,111.599998,51.883759,12.617703,124.075462,176.229797,191.653427,316.636993,146.291382,...,68.378075,745.535156,167.103836,97.083527,15.839675,93.726959,199.918991,94.773384,37.559978,33.397961
2020-07-28,192.759842,3000.330078,110.480003,51.584888,12.548265,125.607742,176.850433,191.414917,316.608002,145.355408,...,68.971642,744.081604,170.468277,95.142258,15.393486,92.120888,198.540359,93.417923,37.432430,33.744724
2020-07-29,192.730377,3033.530029,113.290001,52.401798,12.786336,126.030106,178.909363,189.705505,315.225861,146.958511,...,69.316444,761.355530,172.694901,97.103241,15.335289,93.489029,199.485703,95.663841,38.619667,33.606022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-03,235.559998,3386.489990,198.830002,72.080002,25.677780,135.000000,243.020004,288.929993,379.320007,175.240005,...,76.699997,711.049988,179.960007,158.660004,21.690001,115.459999,350.160004,154.800003,65.760002,45.730000
2021-05-04,233.860001,3311.870117,195.970001,70.820000,25.149881,134.539993,240.000000,302.109985,375.290009,173.770004,...,75.690002,710.030029,180.919998,158.449997,21.570000,114.540001,349.839996,154.529999,66.699997,45.259998
2021-05-05,235.039993,3270.540039,197.550003,71.029999,25.120001,133.460007,239.699997,294.209991,372.500000,173.860001,...,73.779999,693.000000,175.500000,164.339996,22.600000,114.720001,357.619995,156.389999,67.400002,45.439999


In [32]:
# # Convert closing prices to total equity based on number of shares owned
# for index, x in trimmed_senbet.iterrows():
#     portfolio[x.Ticker]*=x.Shares

In [55]:
for sym in portfolio.columns:
    data = list(portfolio[sym])
    pct_change = []
    for i in range(1, len(data)):
        curr = data[i]
        yester = data[i-1]
        delta = curr-yester
        pct_change.append(delta/curr * 100)
    pct_change.insert(0, 0)
    portfolio[sym] = pct_change

portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-24,0.588758,0.743122,0.943911,-0.945358,-1.197139,-0.158783,0.472606,-1.225562,-0.101290,0.103994,...,-0.256924,0.094937,0.017803,-0.983149,-1.954800,0.487956,-0.769351,-1.405640,-0.437465,-1.194042
2020-07-27,1.257146,1.515446,1.272400,-1.555296,1.493717,0.284994,-0.536643,0.565238,0.555556,1.837734,...,-0.943693,2.202917,1.062448,-1.177429,-0.244944,-1.882795,0.763478,0.480215,-1.515155,0.622953
2020-07-28,-2.552992,-1.829128,-1.013754,-0.579375,-0.553367,1.219893,0.350938,-0.124604,-0.009157,-0.643921,...,0.860596,-0.195348,1.973646,-2.040386,-2.898557,-1.743439,-0.694384,-1.450965,-0.340743,1.027608
2020-07-29,-0.015288,1.094433,2.480358,1.558935,1.861913,0.335129,1.150823,-0.901087,-0.438461,1.090855,...,0.497433,2.268838,1.289339,2.019483,-0.379497,1.463424,0.473890,2.347719,3.074177,-0.412731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-03,-0.220752,-2.389788,-1.227180,1.498338,1.861913,1.170372,1.423748,-8.607620,1.906045,1.261131,...,-1.056069,-1.365589,-0.633474,3.044247,7.422778,-1.948727,0.488346,0.936691,0.182486,0.328007
2021-05-04,-0.726929,-2.253104,-1.459407,-1.779161,-2.099011,-0.341911,-1.258335,4.362647,-1.073836,-0.845946,...,-1.334383,-0.143650,0.530617,-0.132538,-0.556332,-0.803211,-0.091472,-0.174726,1.409288,-1.038447
2021-05-05,0.502039,-1.263708,0.799798,0.295648,-0.118951,-0.809221,-0.125158,-2.685155,-0.748996,0.051764,...,-2.588782,-2.457436,-3.088318,3.584033,4.557525,0.156904,2.175493,1.189335,1.038582,0.396127


In [56]:
print(portfolio)
print(asset_weights)

                 MCD      AMZN      FIVE       TJX       DAN        PG  \
Date                                                                     
2020-07-23  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2020-07-24  0.588758  0.743122  0.943911 -0.945358 -1.197139 -0.158783   
2020-07-27  1.257146  1.515446  1.272400 -1.555296  1.493717  0.284994   
2020-07-28 -2.552992 -1.829128 -1.013754 -0.579375 -0.553367  1.219893   
2020-07-29 -0.015288  1.094433  2.480358  1.558935  1.861913  0.335129   
...              ...       ...       ...       ...       ...       ...   
2021-05-03 -0.220752 -2.389788 -1.227180  1.498338  1.861913  1.170372   
2021-05-04 -0.726929 -2.253104 -1.459407 -1.779161 -2.099011 -0.341911   
2021-05-05  0.502039 -1.263708  0.799798  0.295648 -0.118951 -0.809221   
2021-05-06 -0.076638  1.083668 -2.698067  1.017284  3.754787  1.243150   
2021-05-07 -0.008518 -0.448413  2.106870  2.737865  2.393416  0.007395   

                 STZ        EL      C

In [57]:
sharpe_ratio(portfolio.to_numpy(), asset_weights, 0.014)

[[1.02251943]] 0.1281540572175306 0.014


matrix([[0.11163999]])

1.0015